<a href="https://colab.research.google.com/github/razzlestorm/DS-Unit-2-Regression-Classification/blob/master/Jerimiah_Willhite_DS7_Sprint_Challenge_5_Regression_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Regression & Classification Sprint Challenge

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

### Part 1, Classification
- 1.1. Begin with baselines for classification
- 1.2. Do train/test split. Arrange data into X features matrix and y target vector
- 1.3. Use scikit-learn to fit a logistic regression model
- 1.4. Report classification metric: accuracy

### Part 2, Regression
- 2.1. Begin with baselines for regression
- 2.2. Do train/validate/test split. 
- 2.3. Arrange data into X features matrix and y target vector
- 2.4. Do one-hot encoding
- 2.5. Use scikit-learn to fit a linear regression (or ridge regression) model
- 2.6. Report validation MAE and $R^2$

### Stretch Goals, Regression
- Make visualizations to explore relationships between features and target
- Try at least 3 feature combinations. You may select features manually, or automatically
- Report validation MAE and $R^2$ for each feature combination you try
- Report test MAE and $R^2$ for your final model
- Print or plot the coefficients for the features in your model

In [1]:
# If you're in Colab...
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    !pip install category_encoders==2.0.0
    !pip install pandas-profiling==2.3.0
    !pip install plotly==4.1.1

# Part 1, Classification: Predict Blood Donations 🚑
Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict whether the donor made a donation in March 2007, using information about each donor's history.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

In [0]:
import pandas as pd

donors = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')
assert donors.shape == (748,5)

donors = donors.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [3]:
donors.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [4]:
donors.isnull().sum()

months_since_last_donation     0
number_of_donations            0
total_volume_donated           0
months_since_first_donation    0
made_donation_in_march_2007    0
dtype: int64

## 1.1. Begin with baselines

What accuracy score would you get here with a "majority class baseline"?
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [5]:
y_train = donors['made_donation_in_march_2007']
y_train.value_counts(normalize=True)

#baseline = 76.2% did not make a donation (Accuracy of majority class baseline = frequency of the majority class)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

## 1.2. Do train/test split. Arrange data into X features matrix and y target vector

Do these steps in either order.

Split randomly. Use scikit-learn's train/test split function. Include 75% of the data in the train set, and hold out 25% for the test set.

In [6]:

#creating X training data from all columns except donors
X_train = donors.drop(columns=['made_donation_in_march_2007'])
#creating target vector
y_train = donors['made_donation_in_march_2007']

#checking size
X_train.shape, y_train.shape

((748, 4), (748,))

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size = 0.75, test_size = 0.25,
    stratify = y_train, random_state=4242
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((561, 4), (187, 4), (561,), (187,))

## 1.3. Use scikit-learn to fit a logistic regression model

You may use any number of features

In [8]:
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV(n_jobs = -1)

#currently using all the features!
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=-1, penalty='l2',
                     random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

## 1.4. Report classification metric: accuracy

What is your model's accuracy on the test set?

Don't worry if your model doesn't beat the mean baseline. That's okay!

_"The combination of some data and an aching desire for an answer does not ensure that a reasonable answer can be extracted from a given body of data."_ —[John Tukey](https://en.wikiquote.org/wiki/John_Tukey)



In [9]:
model.score(X_val, y_val)

0.7754010695187166

# Part 2, Regression: Predict home prices in Ames, Iowa 🏠

You'll use historical housing data. There's a data dictionary at the bottom of the notebook. 

Run this code cell to load the dataset:





In [0]:
import pandas as pd
URL = 'https://drive.google.com/uc?export=download&id=1522WlEW6HFss36roD_Cd9nybqSuiVcCK'
homes = pd.read_csv(URL)
assert homes.shape == (2904, 47)

In [11]:
homes.isnull().sum()

1st_Flr_SF        0
Bedroom_AbvGr     0
Bldg_Type         0
Bsmt_Full_Bath    0
Bsmt_Half_Bath    0
Central_Air       0
Condition_1       0
Condition_2       0
Electrical        0
Exter_Cond        0
Exter_Qual        0
Exterior_1st      0
Exterior_2nd      0
Foundation        0
Full_Bath         0
Functional        0
Gr_Liv_Area       0
Half_Bath         0
Heating           0
Heating_QC        0
House_Style       0
Kitchen_AbvGr     0
Kitchen_Qual      0
Land_Contour      0
Land_Slope        0
Lot_Area          0
Lot_Config        0
Lot_Shape         0
MS_SubClass       0
MS_Zoning         0
Mas_Vnr_Type      0
Mo_Sold           0
Neighborhood      0
Overall_Cond      0
Overall_Qual      0
Paved_Drive       0
Roof_Matl         0
Roof_Style        0
SalePrice         0
Sale_Condition    0
Sale_Type         0
Street            0
TotRms_AbvGrd     0
Utilities         0
Year_Built        0
Year_Remod/Add    0
Yr_Sold           0
dtype: int64

In [12]:
homes.head()

,1st_Flr_SF,Bedroom_AbvGr,Bldg_Type,Bsmt_Full_Bath,Bsmt_Half_Bath,Central_Air,Condition_1,Condition_2,Electrical,Exter_Cond,Exter_Qual,Exterior_1st,Exterior_2nd,Foundation,Full_Bath,Functional,Gr_Liv_Area,Half_Bath,Heating,Heating_QC,House_Style,Kitchen_AbvGr,Kitchen_Qual,Land_Contour,Land_Slope,Lot_Area,Lot_Config,Lot_Shape,MS_SubClass,MS_Zoning,Mas_Vnr_Type,Mo_Sold,Neighborhood,Overall_Cond,Overall_Qual,Paved_Drive,Roof_Matl,Roof_Style,SalePrice,Sale_Condition,Sale_Type,Street,TotRms_AbvGrd,Utilities,Year_Built,Year_Remod/Add,Yr_Sold
0,1656,3,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,TA,BrkFace,Plywood,CBlock,1,Typ,1656,0,GasA,Fa,1Story,1,TA,Lvl,Gtl,31770,Corner,IR1,20,RL,Stone,5,NAmes,5,6,P,CompShg,Hip,215000,Normal,WD,Pave,7,AllPub,1960,1960,2010
1,896,2,1Fam,0.0,0.0,Y,Feedr,Norm,SBrkr,TA,TA,VinylSd,VinylSd,CBlock,1,Typ,896,0,GasA,TA,1Story,1,TA,Lvl,Gtl,11622,Inside,Reg,20,RH,None,6,NAmes,6,5,Y,CompShg,Gable,105000,Normal,WD,Pave,5,AllPub,1961,1961,2010
2,1329,3,1Fam,0.0,0.0,Y,Norm,Norm,SBrkr,TA,TA,Wd Sdng,Wd Sdng,CBlock,1,Typ,1329,1,GasA,TA,1Story,1,Gd,Lvl,Gtl,14267,Corner,IR1,20,RL,BrkFace,6,NAmes,6,6,Y,CompShg,Hip,172000,Normal,WD,Pave,6,AllPub,1958,1958,2010
3,2110,3,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,Gd,BrkFace,BrkFace,CBlock,2,Typ,2110,1,GasA,Ex,1Story,1,Ex,Lvl,Gtl,11160,Corner,Reg,20,RL,None,4,NAmes,5,7,Y,CompShg,Hip,244000,Normal,WD,Pave,8,AllPub,1968,1968,2010
4,928,3,1Fam,0.0,0.0,Y,Norm,Norm,SBrkr,TA,TA,VinylSd,VinylSd,PConc,2,Typ,1629,1,GasA,Gd,2Story,1,TA,Lvl,Gtl,13830,Inside,IR1,60,RL,None,3,Gilbert,5,5,Y,CompShg,Gable,189900,Normal,WD,Pave,6,AllPub,1997,1998,2010


## 2.1. Begin with baselines

What is the Mean Absolute Error and R^2 score for a mean baseline?

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_train = homes['SalePrice']

majority_class = y_train.mode()[0]

y_pred = [majority_class] * len(y_train)

y_train.value_counts(normalize=True)
#The mode price is 135000
print(y_train.mode()[0])

#the frequency it appears (and thus our accuracy) is 0.011708
print(accuracy_score(y_train, y_pred))

#getting MAE of the baseline
print('Baseline MAE: ', mean_absolute_error(y_train, y_pred))

#getting R^2 score of the baseline
print('Baseline R^2 Score: ', r2_score(y_train, y_pred))

135000
0.011707988980716254
Baseline MAE:  60572.26962809917
Baseline R^2 Score:  -0.3235285881670549


## 2.2. Do train/test split

Train on houses sold in the years 2006 - 2008. (1,920 rows)

Validate on house sold in 2009. (644 rows)

Test on houses sold in 2010. (340 rows)

In [14]:
#Here we do a train/test split based on time!
X_train = homes[(homes['Yr_Sold'] >= 2006) & (homes['Yr_Sold'] <= 2008)]
y_train = X_train['SalePrice']

X_val = homes[homes['Yr_Sold'] == 2009]
y_val = X_val['SalePrice']

X_test = homes[homes['Yr_Sold'] == 2010]
y_test = X_test['SalePrice']

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


((1920, 47), (1920,), (644, 47), (644,), (340, 47), (340,))

In [15]:
#checking out numeric features
X_train.select_dtypes('number').describe().T

,count,mean,std,min,25%,50%,75%,max
1st_Flr_SF,1920.0,1158.817188,391.430679,334.0,886.00,1084.0,1377.25,5095.0
Bedroom_AbvGr,1920.0,2.867188,0.827842,0.0,2.00,3.0,3.00,8.0
Bsmt_Full_Bath,1920.0,0.415625,0.515690,0.0,0.00,0.0,1.00,2.0
Bsmt_Half_Bath,1920.0,0.065625,0.255967,0.0,0.00,0.0,0.00,2.0
Full_Bath,1920.0,1.561458,0.550119,0.0,1.00,2.0,2.00,3.0
Gr_Liv_Area,1920.0,1504.033854,523.332578,334.0,1116.25,1436.0,1753.50,5642.0
Half_Bath,1920.0,0.374479,0.496863,0.0,0.00,0.0,1.00,2.0
Kitchen_AbvGr,1920.0,1.039062,0.201700,0.0,1.00,1.0,1.00,2.0
Lot_Area,1920.0,10257.809896,7807.171357,1470.0,7412.75,9422.0,11601.50,164660.0
MS_SubClass,1920.0,58.192708,43.040262,20.0,20.00,50.0,70.00,190.0


In [16]:
#checking out categorical features
X_train.describe(exclude="number").T.sort_values(by = 'unique')

,count,unique,top,freq
Utilities,1920,2,AllPub,1919
Central_Air,1920,2,Y,1791
Street,1920,2,Pave,1912
Paved_Drive,1920,3,Y,1731
Land_Slope,1920,3,Gtl,1829
Lot_Shape,1920,4,Reg,1203
Land_Contour,1920,4,Lvl,1716
Kitchen_Qual,1920,4,TA,991
Exter_Qual,1920,4,TA,1197
Bldg_Type,1920,5,1Fam,1588


In [17]:
X_train['Bldg_Type'].value_counts(normalize=True)

1Fam      0.827083
TwnhsE    0.081250
Twnhs     0.034375
Duplex    0.033333
2fmCon    0.023958
Name: Bldg_Type, dtype: float64

In [18]:
X_train['Exter_Qual'].value_counts(normalize=True)

TA    0.623437
Gd    0.334375
Ex    0.033333
Fa    0.008854
Name: Exter_Qual, dtype: float64

In [19]:
X_train['Kitchen_Qual'].value_counts(normalize=True)

TA    0.516146
Gd    0.394792
Ex    0.066146
Fa    0.022917
Name: Kitchen_Qual, dtype: float64

In [20]:
X_train['Heating_QC'].value_counts(normalize=True)

Ex    0.508333
TA    0.286458
Gd    0.172917
Fa    0.030729
Po    0.001563
Name: Heating_QC, dtype: float64

## 2.3. Arrange data into X features matrix and y target vector

Select at least one numeric feature and at least one categorical feature.

Otherwise, you many choose whichever features and however many you want.

In [0]:
#This cell tries with originally manually-input/guessed features  

categorical_features = ['Central_Air', 'Bldg_Type', 'Heating_QC', 'Neighborhood', 'Kitchen_Qual', 'Exter_Qual']

numeric_features = X_train[['Lot_Area', 'Overall_Cond', 'Overall_Qual', 'Year_Built', 'Year_Remod/Add', 'Full_Bath']].columns.tolist()

features = categorical_features + numeric_features



#THIS RETURNS:
#Baseline MAE:  23305.58309062403
#Baseline R^2 Score:  0.8373667948032885

## 2.4. Do one-hot encoding

Encode your categorical feature(s).

In [0]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
#manual guess features

X_train_subset = X_train[features]
X_val_subset = X_val[features]
X_test_subset = X_test[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)
X_test_encoded = encoder.transform(X_test_subset)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)



## 2.5. Use scikit-learn to fit a linear regression (or ridge regression) model
Fit your model.

In [0]:
#manual guess features
from sklearn.linear_model import LinearRegression

model = LinearRegression(n_jobs = -1)

model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)
y_pred_val = model.predict(X_val_scaled)
y_pred_test = model.predict(X_test_scaled)



## 2.6. Report validation MAE and $R^2$

What is your model's Mean Absolute Error and $R^2$ score on the validation set?

In [24]:
#getting MAE of model
print('LR Model MAE: ', mean_absolute_error(y_val, y_pred_val))

#getting R^2 score of model
print('LR Model R^2 Score: ', r2_score(y_val, y_pred_val))

LR Model MAE:  23305.58309062403
LR Model R^2 Score:  0.8373667948032885


# **What if we try with ALL features?**

In [0]:
#This cell tries with ALL Features 

all_categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()

all_numeric_features = X_train.select_dtypes(include='number').columns.tolist()

all_features = all_categorical_features + all_numeric_features

#THIS RETURNS (with linear regression):
#Baseline MAE:  173.28199193481282
#Baseline R^2 Score:  0.9984028525751494

In [0]:
#This cell tries with ALL features
X_train_subset = X_train[all_features]
X_val_subset = X_val[all_features]
X_test_subset = X_test[all_features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)
X_test_encoded = encoder.transform(X_test_subset)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [0]:
model = LinearRegression(n_jobs = -1)

model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)
y_pred_val = model.predict(X_val_scaled)
y_pred_test = model.predict(X_test_scaled)


In [28]:
#getting MAE of model_all
print('All-feature LR MAE: ', mean_absolute_error(y_val, y_pred_val))

#getting R^2 score of model_all
print('All-feature LR R^2 Score: ', r2_score(y_val, y_pred_val))

All-feature LR MAE:  173.28199193481282
All-feature LR R^2 Score:  0.9984028525751494


# Stretch Goals, Regression
- Make visualizations to explore relationships between features and target
- Try at least 3 feature combinations. You may select features manually, or automatically
- Report validation MAE and $R^2$ for each feature combination you try
- Report test MAE and $R^2$ for your final model
- Print or plot the coefficients for the features in your model

# **What if we try using KBest to get the BEST features?**

In [29]:
#Using KBest to get features automatically!

from sklearn.feature_selection import f_regression, SelectKBest
for k in range(1, len(X_train_encoded.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $34,044 

2 features
Test MAE: $28,339 

3 features
Test MAE: $26,153 

4 features
Test MAE: $26,058 

5 features
Test MAE: $26,049 

6 features
Test MAE: $24,066 

7 features
Test MAE: $24,064 

8 features
Test MAE: $23,702 

9 features
Test MAE: $23,661 

10 features
Test MAE: $22,653 

11 features
Test MAE: $22,769 

12 features
Test MAE: $22,796 

13 features
Test MAE: $22,791 

14 features
Test MAE: $22,279 

15 features
Test MAE: $22,435 

16 features
Test MAE: $22,401 

17 features
Test MAE: $22,415 

18 features
Test MAE: $22,429 

19 features
Test MAE: $22,485 

20 features
Test MAE: $22,464 

21 features
Test MAE: $22,435 

22 features
Test MAE: $22,009 

23 features
Test MAE: $21,901 

24 features
Test MAE: $21,909 

25 features
Test MAE: $21,925 

26 features
Test MAE: $21,926 

27 features
Test MAE: $21,781 

28 features
Test MAE: $21,859 

29 features
Test MAE: $21,833 

30 features
Test MAE: $21,832 

31 features
Test MAE: $21,964 

32 features
Test 

In [30]:
k = 58
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)

all_names = X_train_encoded.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\nFeatures not selected:')
for name in unselected_names:
    print(name)

Features selected:
Central_Air_Y
Central_Air_N
Electrical_SBrkr
Electrical_FuseA
Exter_Qual_Ex
Exter_Qual_TA
Exter_Qual_Gd
Exterior_1st_VinylSd
Exterior_1st_Wd Sdng
Exterior_2nd_VinylSd
Exterior_2nd_Wd Sdng
Foundation_CBlock
Foundation_PConc
Foundation_BrkTil
Heating_QC_Ex
Heating_QC_TA
House_Style_2Story
House_Style_1.5Fin
Kitchen_Qual_TA
Kitchen_Qual_Gd
Kitchen_Qual_Ex
Land_Contour_HLS
Lot_Config_CulDSac
Lot_Shape_IR1
Lot_Shape_Reg
MS_Zoning_RL
MS_Zoning_RM
Mas_Vnr_Type_None
Mas_Vnr_Type_BrkFace
Mas_Vnr_Type_Stone
Neighborhood_NAmes
Neighborhood_StoneBr
Neighborhood_NridgHt
Neighborhood_NoRidge
Neighborhood_Somerst
Neighborhood_BrkSide
Neighborhood_OldTown
Neighborhood_IDOTRR
Neighborhood_Edwards
Paved_Drive_Y
Paved_Drive_N
Roof_Style_Hip
Roof_Style_Gable
Sale_Condition_Normal
Sale_Condition_Partial
Sale_Type_WD 
Sale_Type_New
1st_Flr_SF
Bedroom_AbvGr
Bsmt_Full_Bath
Full_Bath
Gr_Liv_Area
Half_Bath
Lot_Area
Overall_Qual
TotRms_AbvGrd
Year_Built
Year_Remod/Add

Features not selected:
B

In [0]:
#USING KBEST TO MANUALLY UPDATE FEATURES USED:

Kcategorical_features = ['Central_Air', 'Electrical','Exter_Qual',
                        'Exterior_1st', 'Exterior_2nd', 'Foundation', 'Heating_QC',
                        'House_Style', 'Kitchen_Qual', 'Land_Contour', 'Lot_Config',
                        'Lot_Shape', 'MS_Zoning', 'Mas_Vnr_Type', 'Neighborhood',
                        'Paved_Drive', 'Roof_Style', 'Sale_Condition', 'Sale_Type',
                        ]

Knumeric_features = X_train[
                           ['1st_Flr_SF', 'Bedroom_AbvGr', 'Bsmt_Full_Bath', 
                            'Full_Bath', 'Gr_Liv_Area', 'Half_Bath', 'Lot_Area',
                            'Overall_Qual', 'TotRms_AbvGrd', 'Year_Built', 
                            'Year_Remod/Add']].columns.tolist()

Kfeatures = Kcategorical_features + Knumeric_features


In [0]:
#Using KBest features
X_train_subset = X_train[Kfeatures]
X_val_subset = X_val[Kfeatures]
X_test_subset = X_test[Kfeatures]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)
X_test_encoded = encoder.transform(X_test_subset)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


In [0]:
#Now we use RidgeCV
from sklearn.linear_model import RidgeCV

model = RidgeCV()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)
y_pred_val = model.predict(X_val_scaled)
y_pred_test = model.predict(X_test_scaled)

In [34]:
#getting MAE of model_k
print('RidgeCV MAE: ', mean_absolute_error(y_val, y_pred_val))

#getting R^2 score of model_k
print('RidgeCV R^2 Score: ', r2_score(y_val, y_pred_val))

RidgeCV MAE:  19624.925396476676
RidgeCV R^2 Score:  0.8708168638087197


# Data Dictionary 

Here's a description of the data fields:

```
1st_Flr_SF: First Floor square feet

Bedroom_AbvGr: Bedrooms above grade (does NOT include basement bedrooms)

Bldg_Type: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
       
Bsmt_Half_Bath: Basement half bathrooms

Bsmt_Full_Bath: Basement full bathrooms

Central_Air: Central air conditioning

       N	No
       Y	Yes
		
Condition_1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition_2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
       
 Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
       
 Exter_Cond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
 
 Exter_Qual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Exterior_1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior_2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
       
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
Full_Bath: Full bathrooms above grade

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Gr_Liv_Area: Above grade (ground) living area square feet
        
Half_Bath: Half baths above grade

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
Heating_QC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

House_Style: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

Kitchen_AbvGr: Kitchens above grade

Kitchen_Qual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Land_Slope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

Lot_Area: Lot size in square feet

Lot_Config: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

Lot_Shape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular

MS_SubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MS_Zoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

Mas_Vnr_Type: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone

Mo_Sold: Month Sold (MM)

Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Overall_Cond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

Overall_Qual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

Paved_Drive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel

Roof_Matl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

Roof_Style: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

SalePrice: the sales price for each house

Sale_Condition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

Sale_Type: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
	
Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
TotRms_AbvGrd: Total rooms above grade (does not include bathrooms)

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
Year_Built: Original construction date

Year_Remod/Add: Remodel date (same as construction date if no remodeling or additions)
						
Yr_Sold: Year Sold (YYYY)	

```